# CamelAI + TiDB Demo

Here is a demo show you how to use CamelAI agent with the memory storage capabilities provided by TiDB.

### Install dependencies

In [ ]:
!rm -rf .venv/lib/python3.11/site-packages/camel*
!uv cache clean camel-ai
!uv add ../camel
!uv sync

### Load the environment variables

1. Go to [TiDB Cloud](https://tidbcloud.com/console/clusters)
2. Create a serverless cluster in a few seconds
3. Setup the environment variables, add the tidb database url and OpenAI API Key to the `.env` file:

In [3]:
# cp .env.example .env
# vi .env

True

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

### Initialize the memory with TiDB

In [ ]:
from camel.memories import (
    ChatHistoryBlock,
    LongtermAgentMemory,
    ScoreBasedContextCreator,
    VectorDBBlock,
)
from camel.types import ModelType, OpenAIBackendRole
from camel.utils import OpenAITokenCounter
from camel.embeddings import OpenAIEmbedding
from camel.storages import TiDBStorage

embedding =  OpenAIEmbedding()
memory = LongtermAgentMemory(
    context_creator=ScoreBasedContextCreator(
        token_counter=OpenAITokenCounter(ModelType.GPT_4O_MINI),
        token_limit=1024,
    ),
    chat_history_block=ChatHistoryBlock(),
    vector_db_block=VectorDBBlock(
        storage=TiDBStorage(
            url_and_api_key=os.environ["DATABASE_URL"],
            vector_dim=embedding.get_output_dim(),
            collection_name="camel_ai",
        )
    ),
)

### Add records to the memory

In [ ]:
from camel.messages import BaseMessage
from camel.memories import MemoryRecord

records = [
    MemoryRecord(
        message=BaseMessage.make_user_message(
            role_name="User",
            meta_dict=None,
            content="What is CAMEL AI?",
        ),
        role_at_backend=OpenAIBackendRole.USER,
    ),
    MemoryRecord(
        message=BaseMessage.make_assistant_message(
            role_name="Agent",
            meta_dict=None,
            content="CAMEL-AI.org is the 1st LLM multi-agent framework and "
                    "an open-source community dedicated to finding the scaling law "
                    "of agents.",
        ),
        role_at_backend=OpenAIBackendRole.ASSISTANT,
    ),
]
memory.write_records(records)

### Retrieve the context from the memory

In [ ]:
context, token_count = memory.get_context()

print(f"Retrieved context (token count: {token_count}):")
for message in context:
    print(f"{message}")

### Chat with the agent

In [ ]:
from camel.agents import ChatAgent

# Define system message for the agent
sys_msg = BaseMessage.make_assistant_message(
    role_name='Agent',
    content='You are a curious agent wondering about the universe.',
)

# Initialize agent
agent = ChatAgent(system_message=sys_msg)
agent.memory = memory  # 👈 Add the memory to the agent.

# Define a user message
usr_msg = BaseMessage.make_user_message(
    role_name='User',
    content="Tell me which is the 1st LLM multi-agent framework based on what we have discussed",
)

# Sending the message to the agent
response = agent.step(usr_msg)

# Check the response (just for illustrative purpose)
print(response.msgs[0].content)